In [2]:
import pandas as pd
import numpy as np

import yfinance as yf


import requests
from datetime import datetime, timedelta, timezone


import warnings

warnings.filterwarnings('ignore')

MONTH_DIR = 'Jul 2025'


/Users/marc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
# Stables
# 31566704 USDC
# 312769 USDT 
# 227855942 EURS
# 760037151 xUSD


In [4]:
import pandas as pd
import requests
from io import StringIO

def download_csv(coin_id, currency):
    """
    This function downloads the data from Coingecko and returns it as a DataFrame.
    Inputs:
        - coin_id: the name of the crypto we are getting the data
        - currency: by default it is the usd
    Output:
        - df: pandas DataFrame with the historical data
    """
    # Construct the URL
    base_url = "https://www.coingecko.com"
    csv_path = f"/price_charts/export/{coin_id}/{currency}.csv"
    full_url = base_url + csv_path

    # Download the CSV
    response = requests.get(full_url)
    if response.status_code == 200:
        # Create DataFrame directly from response content
        df = pd.read_csv(StringIO(response.content.decode('utf-8')))
        print(f"Successfully loaded data for {coin_id}.")
        return df
    else:
        print(f"Failed to download CSV for {coin_id}.")
        return None

df = download_csv('algorand', 'usd')


Successfully loaded data for algorand.


In [12]:
df['snapped_at'] = pd.to_datetime(df['snapped_at']).dt.date

df['VWAP'] = (df['price'] * df['total_volume']).cumsum() / df['total_volume'].cumsum()
df_quarter_ends = df[pd.to_datetime(df['snapped_at']).dt.is_quarter_end]
df_quarter_ends #.to_csv('algorand_vwap.csv', index=False)

,snapped_at,price,market_cap,total_volume,VWAP
9,2019-06-30,1.506861,1.063096e+08,1.287885e+08,1.842996
101,2019-09-30,0.224723,9.541233e+07,6.963564e+07,0.894450
193,2019-12-31,0.223841,1.180638e+08,3.709186e+07,0.572997
284,2020-03-31,0.153444,1.069825e+08,5.899616e+07,0.488462
375,2020-06-30,0.212518,1.707294e+08,6.156709e+07,0.436335
467,2020-09-30,0.334986,2.682717e+08,9.324964e+07,0.438907
559,2020-12-31,0.335254,2.693051e+08,1.233747e+08,0.416509
649,2021-03-31,1.325524,3.464285e+09,5.342558e+08,0.709574
740,2021-06-30,0.882501,2.752411e+09,2.286890e+08,0.855958
832,2021-09-30,1.599053,9.548035e+09,4.427825e+08,1.031568


In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime

algousd = yf.download('ALGO-USD', start='2025-01-01', end=datetime.now(), progress=False, auto_adjust=True)
# Handle multi-level columns from yfinance
if isinstance(algousd.columns, pd.MultiIndex):
    algousd.columns = algousd.columns.get_level_values(0)

# Create a DataFrame with just the close prices
price_df = pd.DataFrame({
    'date': algousd.index,
    'open': algousd['Open'].values,
    'high': algousd['High'].values,
    'low': algousd['Low'].values,
    'close': algousd['Close'].values,
    'volume': algousd['Volume'].values
})

price_df['avg_price'] = (price_df['high'] + price_df['low'] + price_df['close'] + price_df['open']) / 4
price_df['VWAP'] = (price_df['avg_price'] * price_df['volume']).cumsum() / price_df['volume'].cumsum()

price_df

,date,open,high,low,close,volume,avg_price,VWAP
0,2025-01-01,0.333701,0.384794,0.327296,0.376270,293751245,0.355515,0.355515
1,2025-01-02,0.376269,0.406824,0.376243,0.392243,355149024,0.387895,0.373237
2,2025-01-03,0.392250,0.425831,0.392197,0.418233,362071902,0.407128,0.385375
3,2025-01-04,0.418233,0.430344,0.405990,0.409769,247562926,0.416084,0.391415
4,2025-01-05,0.409769,0.415470,0.396724,0.405151,220267450,0.406778,0.393704
...,...,...,...,...,...,...,...,...
326,2025-11-23,0.135694,0.145959,0.135242,0.143644,51053812,0.140135,0.270429
327,2025-11-24,0.143655,0.147116,0.139240,0.143671,64009925,0.143421,0.270227
328,2025-11-25,0.143671,0.147343,0.140138,0.146376,53534086,0.144382,0.270060
329,2025-11-26,0.146376,0.148253,0.141681,0.146284,50693911,0.145648,0.269904


In [10]:
price_df['date'] = pd.to_datetime(price_df['date'])
price_df['quarter'] = price_df['date'].dt.to_period('Q')
price_df['avg_price'] = (price_df['high'] + price_df['low'] + price_df['close'] + price_df['open']) / 4

price_df['pv'] = price_df['avg_price'] * price_df['volume']
price_df['cumulative_pv'] = price_df.groupby('quarter')['pv'].cumsum()
price_df['cumulative_volume'] = price_df.groupby('quarter')['volume'].cumsum()

# Calculate VWAP
price_df['vwap'] = price_df['cumulative_pv'] / price_df['cumulative_volume']


price_df_quarter_ends = price_df[pd.to_datetime(price_df['date']).dt.is_quarter_end]
price_df_quarter_ends

,date,open,high,low,close,volume,avg_price,VWAP,quarter,pv,cumulative_pv,cumulative_volume,vwap
89,2025-03-31,0.179389,0.181528,0.172695,0.180097,70773544,0.178427,0.337150,2025Q1,1.262793e+07,5.600097e+09,16610081996,0.337150
180,2025-06-30,0.185949,0.186967,0.177142,0.185459,64516672,0.183879,0.295993,2025Q2,1.186328e+07,1.433571e+09,7152846809,0.200420
272,2025-09-30,0.207763,0.210226,0.201663,0.207899,107192472,0.206888,0.282523,2025Q3,2.217681e+07,2.908630e+09,11428128235,0.254515


In [11]:
def date_to_unix_timestamp(start_date_str, end_date_str):
  """
  Converts start and end dates in YYYY-MM-DD format to Unix timestamps.

  Args:
    start_date_str: Start date string in YYYY-MM-DD format.
    end_date_str: End date string in YYYY-MM-DD format.

  Returns:
    A tuple containing the start and end Unix timestamps.
  """

  start_date = datetime.strptime(start_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
  end_date = datetime.strptime(end_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(days=1) - timedelta(seconds=1) 

  start_timestamp = int(start_date.timestamp())
  end_timestamp = int(end_date.timestamp())

  return start_timestamp, end_timestamp

In [12]:
def get_close_price(start_date, end_date, asset_id):
    """
    Fetches an asset historical data from the API for the given interval.
    
    Args:
        start_timestamp: start unix timestamp
        end_timestamp: end unix timestamp

    Returns:
        pd.DataFrame: Dataframe containing the fetched data in daily intervals.
    """
    start_unix, end_unix = date_to_unix_timestamp(start_date, end_date)

    price_feed = f'https://indexer.vestige.fi/assets/{asset_id}/candles?network_id=0&interval=86400&start={start_unix}&end={end_unix}&denominating_asset_id=0&volume_in_denominating_asset=false'

    response = requests.get(price_feed)
    data = response.json()
    df = pd.DataFrame(data)
    return df

In [16]:
start_date = '2025-07-01'
end_date = '2025-07-31'

stables_dfs = {}
stables_ids = {'usdc': 31566704, 'usdt': 312769,
              'eurs': 227855942,'xusd':760037151}

for stablecoin, stables_id in stables_ids.items():
    stables_dfs[stablecoin] = get_close_price(start_date, end_date, stables_id)


stables_dfs

{'usdc':     network_id  asset_id  interval  denominating_asset_id   timestamp    open  \
 0            0  31566704     86400                      0  1751328000  5.4147   
 1            0  31566704     86400                      0  1751414400  5.8315   
 2            0  31566704     86400                      0  1751500800  5.5437   
 3            0  31566704     86400                      0  1751587200  5.4727   
 4            0  31566704     86400                      0  1751673600  5.7479   
 5            0  31566704     86400                      0  1751760000  5.7093   
 6            0  31566704     86400                      0  1751846400  5.6397   
 7            0  31566704     86400                      0  1751932800  5.7083   
 8            0  31566704     86400                      0  1752019200  5.5708   
 9            0  31566704     86400                      0  1752105600  5.1857   
 10           0  31566704     86400                      0  1752192000  4.7472   
 11     

In [33]:
start_date = '2025-07-01'
end_date = '2025-07-31'

asas_dfs = {}
asset_ids = {
    'ora': 1284444444, 'talgo': 2537013734, 'vote': 452399768, 'gobtc': 386192725, 
    'opul': 287867876, 'tiny': 2200000000, 'monko': 2494786278, 'niko': 1265975021, 
    'goeth': 386195940, 'alpha': 2726252423, 'gora': 1138500612, 'deathos': 957484854, 
    'daffir': 1268830233, 'shame': 1035301184, 'stbl': 465865291, 'golddao': 1241945177, 
    'malgo': 1185173782, 'coop': 796425061, 'xalgo': 1134696561, 'compx': 1732165149
}

for asa, asa_id in asset_ids.items():
    asas_dfs[asa] = get_close_price(start_date, end_date, asa_id)

In [34]:
def create_combined_df(dataframes, df_name):
  """
  Creates a single DataFrame with 'date' as the index and 
  'close' columns named after the stablecoin keys.

  Args:
    dataframes: A dictionary where keys are stablecoin names 
                and values are the corresponding DataFrames.

  Returns:
    A pandas DataFrame with the combined data.
  """

  combined_df = pd.DataFrame()
  if df_name == 'stables':
    for asset, df in dataframes.items():
      df['date'] = pd.to_datetime(df['timestamp'], unit='s') 
      df.set_index('date', inplace=True) 
      df.rename(columns={'close': asset}, inplace=True) 
      combined_df = combined_df.join(df[asset], how='outer') 

    return combined_df
  else:
    for asset, df in dataframes.items():
      df['date'] = pd.to_datetime(df['timestamp'], unit='s') 
      df.set_index('date', inplace=True) 
      df.rename(columns={'close': asset}, inplace=True) 
      combined_df = combined_df.join(df[asset], how='outer') 

    return combined_df

# Example usage
stables_final_df = create_combined_df(stables_dfs, 'stables') 
asas_final_df = create_combined_df(asas_dfs, 'asas')

In [35]:
stables_final_df.to_csv(f'{MONTH_DIR}/Results/stablecoins_25.csv')

In [36]:
asas_final_df.to_csv(f'{MONTH_DIR}/Results/asas_jul.csv')

In [37]:
algo_price = pd.read_csv(f'{MONTH_DIR}/algorand_historical_data.csv')
algo_price['snapped_at'] = pd.to_datetime(algo_price['snapped_at'])
price_feed = algo_price[(algo_price['snapped_at'] >= start_date) & (algo_price['snapped_at'] <= end_date)]
price_feed[['snapped_at', 'price']].to_csv(f'{MONTH_DIR}/Results/ALGO_jul.csv', index=False)

In [38]:
asa_volumes = pd.read_csv(f'{MONTH_DIR}/asa_volumes.csv')
asa_volumes['dt'] = pd.to_datetime(asa_volumes['dt'])
asa_volumes.columns.values[1:]
asas_final_df.reset_index(inplace=True)
asas_final_df['date'] = pd.to_datetime(asas_final_df['date'])

In [42]:
asa_volumes = pd.read_csv(f'{MONTH_DIR}/asa_volumes.csv')
asa_volumes['dt'] = pd.to_datetime(asa_volumes['dt'])


def multiply_columns(df1, df2):
  """
  Joins price and volume dataframes to convert into ALGO volumes

  Args:
    df1: Volumes Dataframe in native Currency
    df2: Asset/ALGO Price DataFrame.

  Returns:
    A merged DataFrame with ALGO Volumes
  """

  # Merge DataFrames on 'dt' and 'date'
  merged_df = df1.merge(df2, left_on='dt', right_on='date', suffixes=('_df1', '_df2'))

  # Calculate product of corresponding columns
  columns_to_multiply = df1.columns.values[1:]
  for col in columns_to_multiply:
      merged_df[col] = merged_df[col + '_df1'] * merged_df[col + '_df2']

  # Drop unnecessary columns
  merged_df.drop(columns=[col + '_df1' for col in columns_to_multiply] + 
                  [col + '_df2' for col in columns_to_multiply], 
             inplace=True)

  # Rename 'dt' column to 'date'
  merged_df.rename(columns={'dt': 'date'}, inplace=True)

  return merged_df


result_df = multiply_columns(asa_volumes, asas_final_df)

result_df.to_csv(f'{MONTH_DIR}/Results/asas_algo_volumes.csv', index=False)

In [43]:
def price_merge(algo_volumes, algo_price):
  """
  Joins volumes and algo price dataframes to convert into USD volume

  Args:
    df1: Volumes Dataframe in ALGO
    df2: ALGO/USD Price DataFrame.

  Returns:
    A merged DataFrame with ALGO Volumes
  """

    # Merge DataFrames on 'dt' and 'date'
  merged_df = algo_volumes.merge(algo_price[['snapped_at', 'price']], left_on='date', right_on='snapped_at')

  merged_df.drop('snapped_at', axis=1, inplace=True)
  # Calculate product of corresponding columns
  merged_df['usd_vol'] = merged_df['algo_vol']*merged_df['price']

  # Rename 'dt' column to 'date'
  #merged_df.rename(columns={'dt': 'date'}, inplace=True)

  return merged_df


stables = pd.read_csv(f'{MONTH_DIR}/stables.csv')
algorand = pd.read_csv(f'{MONTH_DIR}/algorand_historical_data.csv')

stables['dt'] = pd.to_datetime(stables['dt'])
algorand['snapped_at'] = pd.to_datetime(algorand['snapped_at']).dt.date

result_df = multiply_columns(stables, stables_final_df)
result_df['algo_vol'] = result_df[result_df.columns.values[1:]].sum(axis=1)

result_df['date'] = pd.to_datetime(result_df['date']).dt.date

usd = price_merge(result_df, algorand)


In [44]:
result_df.to_csv(f'{MONTH_DIR}/Results/stables_algo_volumes.csv', index=False)

In [45]:
# HAFN

def get_price_feed(ticker):
  """
  Fetches the price feed for the specified ticker for December 2024 
  using the yfinance library.

  Args:
    ticker: The ticker symbol (e.g., "AFNUSD=X").

  Returns:
    A pandas DataFrame containing the historical data for the specified period.
  """

  try:
    # Define start and end dates, consider forex does not operate 24/7
    start_date = "2025-07-01"
    end_date = "2025-07-31"

    # Download historical data using yfinance
    data = yf.download(ticker, start=start_date, end=end_date)
    return data
  except Exception as e:
    print(f"Error fetching data for {ticker}: {e}")
    return None

# Example usage
ticker = "AFNUSD=X"
december_data = get_price_feed(ticker)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AFNUSD=X']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [25]:
december_data.to_csv(f'{MONTH_DIR}/Results/HAFN_dec.csv')